# Metrics for RPN and GCViT


In [1]:
from torch import Tensor
import torch
import pandas as pd
from project.dataset import Dataset, VALDODataset
from project.preprocessing import NiftiToTensorTransform
from torch.utils.data import DataLoader
from project.utils import collatev2, compute_statistics

d:\Programming Files\Python\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
INFO:albumentations.check_version:A new version of Albumentations is available: 1.4.21 (you have 1.4.7). Upgrade using: pip install --upgrade albumentations


In [2]:
ds = Dataset()

data = pd.read_csv('targets.csv')
data.shape

(7986, 7)

In [3]:
data = data.query('has_microbleed_slice == 1').reset_index(drop=True)

In [4]:
from sklearn.model_selection import train_test_split

def make_loaders(data,
                 cohort,
                 batch_size,
                 test_size=0.2,
                 random_state=12,
                 target_shape=(300, 300),
                 rpn_mode=True,
                 logger=None
                ):
    data = data[data.cohort == cohort]
    
    s = f'Creating loaders for Cohort {cohort}\n'

    data_train, data_test = train_test_split(
        data,
        test_size=test_size,
        random_state=random_state
    )

    s += f'TRAIN & TEST: {data_train.shape, data_test.shape}\n'

    paths = data_train.mri.unique().tolist()
    s += f'Total Unique MRI Samples in data_train: {len(paths)}\n'
    
    global_min, global_max = compute_statistics(paths)
    s += f'GLOBAL MIN & MAX {global_min, global_max}\n'

    transform = NiftiToTensorTransform(
        target_shape=target_shape,
        rpn_mode=rpn_mode,
        normalization=(global_min, global_max)
    )

    train_set = VALDODataset(
        cases=data_train.mri.tolist(),
        masks=data_train.masks.tolist(),
        target=data_train.target.tolist(),
        transform=transform
    )
    val_set = VALDODataset(
        cases=data_test.mri.tolist(),
        masks=data_test.masks.tolist(),
        target=data_test.target.tolist(),
        transform=transform
    )

    if logger != None:
        logger.info(s)
    else:
        print(s)
    
    return train_set, val_set

In [5]:
t2, v2 = make_loaders(
    data=data,
    cohort=2,
    batch_size=4
)

pixdim[0] (qfac) should be 1 (default) or -1; setting qfac to 1
INFO:nibabel.global:pixdim[0] (qfac) should be 1 (default) or -1; setting qfac to 1
pixdim[0] (qfac) should be 1 (default) or -1; setting qfac to 1
INFO:nibabel.global:pixdim[0] (qfac) should be 1 (default) or -1; setting qfac to 1
pixdim[0] (qfac) should be 1 (default) or -1; setting qfac to 1
INFO:nibabel.global:pixdim[0] (qfac) should be 1 (default) or -1; setting qfac to 1
pixdim[0] (qfac) should be 1 (default) or -1; setting qfac to 1
INFO:nibabel.global:pixdim[0] (qfac) should be 1 (default) or -1; setting qfac to 1
pixdim[0] (qfac) should be 1 (default) or -1; setting qfac to 1
INFO:nibabel.global:pixdim[0] (qfac) should be 1 (default) or -1; setting qfac to 1
pixdim[0] (qfac) should be 1 (default) or -1; setting qfac to 1
INFO:nibabel.global:pixdim[0] (qfac) should be 1 (default) or -1; setting qfac to 1
pixdim[0] (qfac) should be 1 (default) or -1; setting qfac to 1
INFO:nibabel.global:pixdim[0] (qfac) should be 1

Creating loaders for Cohort 2
TRAIN & TEST: ((217, 7), (55, 7))
Total Unique MRI Samples in data_train: 15
GLOBAL MIN & MAX (0.0, 88731.5390625)



In [6]:
tl = DataLoader(
    t2,
    shuffle=True,
    batch_size=20,
    collate_fn=collatev2
)
vl = DataLoader(
    v2,
    shuffle=True,
    batch_size=20,
    collate_fn=collatev2
)


## RPN metrics


In [7]:
from project.model import RPN

device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

rpn_model = RPN(
    input_dim=512,
    output_dim=4,
    image_size=300,
    nh=4,
    pretrained=True
).to(device)

d:\Programming Files\Python\Lib\site-packages\torch\nn\modules\transformer.py:306: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")


In [8]:
rpn_model.load_state_dict(torch.load('weights/RPN_cohorts_1_3_50_epochs_mse_loss_Nov_06_2024_205918.pt'))

<All keys matched successfully>

### IOU


In [9]:
from torchmetrics.functional.detection import intersection_over_union
from torchvision.ops import box_area
from typing import Tuple


In [10]:
def _upcast(t: Tensor) -> Tensor:
    # Protects from numerical overflows in multiplications by upcasting to the equivalent higher type
    if t.is_floating_point():
        return t if t.dtype in (torch.float32, torch.float64) else t.float()
    else:
        return t if t.dtype in (torch.int32, torch.int64) else t.int()

def _box_inter_union(boxes1: Tensor, boxes2: Tensor) -> Tuple[Tensor, Tensor]:
    area1 = box_area(boxes1)
    area2 = box_area(boxes2)

    lt = torch.max(boxes1[:, None, :2], boxes2[:, :2])  # [N,M,2]
    rb = torch.min(boxes1[:, None, 2:], boxes2[:, 2:])  # [N,M,2]

    wh = _upcast(rb - lt).clamp(min=0)  # [N,M,2]
    inter = wh[:, :, 0] * wh[:, :, 1]  # [N,M]

    union = area1[:, None] + area2 - inter

    return inter, union

In [11]:
image_size = 300

## Train Data Loader


In [12]:
tl_iou_scores = []
tl_preds = []
tl_truth = []
tl_cases = []
tl_targets = []
tl_precision_scores = []
tl_recall_scores = []
tl_f1_scores = []
tl_counter = 0

with torch.no_grad():
    for batch in tl:
        for i in range(len(batch)):
            slices, masks, target, case = batch[i]
            x = slices.squeeze(1).repeat(1, 3, 1, 1).float().to(device)
            T = masks.squeeze(1).float().to(device)/image_size

            y = rpn_model(x, target)

            tl_preds.append((y *  image_size).squeeze().detach().cpu().numpy())
            tl_truth.append((T[target] * image_size).squeeze().detach().cpu().numpy())
            tl_cases.append(case)
            tl_targets.append(target)

            tl_iou_score = intersection_over_union(y * image_size, T[target] * image_size)

            tl_inter, tl_union = _box_inter_union(y * image_size, T[target] * image_size)

            tl_precision_score = (tl_inter / box_area(y * image_size))
            tl_recall_score = (tl_inter / box_area(T[target] * image_size))

            if any([tl_precision_score, tl_recall_score]) == 0:
                tl_f1_score = 0
                tl_f1_scores.append(tl_f1_score)
            else:
                tl_f1_score = (2 * (tl_precision_score * tl_recall_score)) / (tl_precision_score + tl_recall_score)
                tl_f1_scores.append(tl_f1_score.detach().cpu().numpy())

            tl_iou_scores.append(tl_iou_score.detach().cpu().numpy())
            tl_precision_scores.append(tl_precision_score.squeeze().squeeze().detach().cpu().numpy())
            tl_recall_scores.append(tl_recall_score.squeeze().squeeze().detach().cpu().numpy())
            tl_counter += 1


pixdim[0] (qfac) should be 1 (default) or -1; setting qfac to 1
INFO:nibabel.global:pixdim[0] (qfac) should be 1 (default) or -1; setting qfac to 1
pixdim[0] (qfac) should be 1 (default) or -1; setting qfac to 1
INFO:nibabel.global:pixdim[0] (qfac) should be 1 (default) or -1; setting qfac to 1
pixdim[0] (qfac) should be 1 (default) or -1; setting qfac to 1
INFO:nibabel.global:pixdim[0] (qfac) should be 1 (default) or -1; setting qfac to 1
pixdim[0] (qfac) should be 1 (default) or -1; setting qfac to 1
INFO:nibabel.global:pixdim[0] (qfac) should be 1 (default) or -1; setting qfac to 1
pixdim[0] (qfac) should be 1 (default) or -1; setting qfac to 1
INFO:nibabel.global:pixdim[0] (qfac) should be 1 (default) or -1; setting qfac to 1
pixdim[0] (qfac) should be 1 (default) or -1; setting qfac to 1
INFO:nibabel.global:pixdim[0] (qfac) should be 1 (default) or -1; setting qfac to 1
pixdim[0] (qfac) should be 1 (default) or -1; setting qfac to 1
INFO:nibabel.global:pixdim[0] (qfac) should be 1

In [13]:
tl_score_dict = {
    'Case': tl_cases,
    'Target': tl_targets,
    'Ground_Truth': tl_truth,
    'Predictions': tl_preds,
    'IOU': tl_iou_scores,
    'Precision': tl_precision_scores,
    'Recall': tl_recall_scores,
    'F1': tl_f1_scores
}

tl_score_df = pd.DataFrame(tl_score_dict)

In [14]:
tl_score_df

,Case,Target,Ground_Truth,Predictions,IOU,Precision,Recall,F1
0,d:\Github\Thesis and ML Project\Dataset\VALDO_...,94,"[108.39844, 83.203125, 168.16406, 143.55469]","[86.08783, 105.86236, 124.67617, 177.86644]",0.10629835,0.22081758,0.17010102,[[0.19216938]]
1,d:\Github\Thesis and ML Project\Dataset\VALDO_...,80,"[36.914062, 164.64844, 97.85156, 226.17188]","[69.33772, 130.81969, 131.77827, 198.7461]",0.13853358,0.2292318,0.25933135,[[0.24335438]]
2,d:\Github\Thesis and ML Project\Dataset\VALDO_...,127,"[70.3125, 162.89062, 130.66406, 223.24219]","[79.415985, 110.77948, 141.1785, 164.07048]",0.008797217,0.018370777,0.01660081,[[0.017441003]]
3,d:\Github\Thesis and ML Project\Dataset\VALDO_...,102,"[63.867188, 108.984375, 188.67188, 206.25]","[63.53385, 111.32702, 151.20926, 182.82991]",0.5134581,0.99619806,0.51446617,[[0.67852294]]
4,d:\Github\Thesis and ML Project\Dataset\VALDO_...,119,"[42.1875, 109.57031, 152.92969, 248.4375]","[79.46232, 86.16796, 152.05502, 173.55537]",0.27198967,0.73219997,0.30203608,[[0.4276602]]
...,...,...,...,...,...,...,...,...
212,d:\Github\Thesis and ML Project\Dataset\VALDO_...,98,"[122.46094, 130.07812, 182.8125, 191.60156]","[66.29293, 97.66219, 138.5808, 194.69334]",0.101869754,0.14139213,0.26709926,[[0.18490346]]
213,d:\Github\Thesis and ML Project\Dataset\VALDO_...,111,"[169.33594, 103.71094, 229.10156, 163.47656]","[79.73, 105.02988, 154.25197, 171.5986]",0.0,0.0,0.0,0
214,d:\Github\Thesis and ML Project\Dataset\VALDO_...,42,"[60.351562, 142.38281, 120.11719, 202.14844]","[113.477295, 157.00975, 162.54918, 159.3838]",0.004292104,0.13530952,0.0044131503,[[0.00854752]]
215,d:\Github\Thesis and ML Project\Dataset\VALDO_...,109,"[41.015625, 127.14844, 220.89844, 194.53125]","[81.93103, 131.40921, 114.56304, 190.11206]",0.15803899,1.0,0.15803899,[[0.27294242]]


## Validation Data Loader


In [15]:
vl_iou_scores = []
vl_preds = []
vl_truth = []
vl_cases = []
vl_targets = []
vl_precision_scores = []
vl_recall_scores = []
vl_f1_scores = []
vl_counter = 0

with torch.no_grad():
    for batch in vl:
        for i in range(len(batch)):
            slices, masks, target, case = batch[i]
            x = slices.squeeze(1).repeat(1, 3, 1, 1).float().to(device)
            T = masks.squeeze(1).float().to(device)/image_size

            y = rpn_model(x, target)

            vl_preds.append((y *  image_size).squeeze().detach().cpu().numpy())
            vl_truth.append((T[target] * image_size).squeeze().detach().cpu().numpy())
            vl_cases.append(case)
            vl_targets.append(target)

            vl_iou_score = intersection_over_union(y * image_size, T[target] * image_size)

            vl_inter, vl_union = _box_inter_union(y * image_size, T[target] * image_size)

            vl_precision_score = (vl_inter / box_area(y * image_size))
            vl_recall_score = (vl_inter / box_area(T[target] * image_size))

            if any([vl_precision_score, vl_recall_score]) == 0:
                vl_f1_score = 0
                vl_f1_scores.append(vl_f1_score)
            else:
                vl_f1_score = (2 * (vl_precision_score * vl_recall_score)) / (vl_precision_score + vl_recall_score)
                vl_f1_scores.append(vl_f1_score.detach().cpu().numpy())

            vl_iou_scores.append(vl_iou_score.detach().cpu().numpy())
            vl_precision_scores.append(vl_precision_score.squeeze().squeeze().detach().cpu().numpy())
            vl_recall_scores.append(vl_recall_score.squeeze().squeeze().detach().cpu().numpy())
            vl_counter += 1


pixdim[0] (qfac) should be 1 (default) or -1; setting qfac to 1
INFO:nibabel.global:pixdim[0] (qfac) should be 1 (default) or -1; setting qfac to 1


pixdim[0] (qfac) should be 1 (default) or -1; setting qfac to 1
INFO:nibabel.global:pixdim[0] (qfac) should be 1 (default) or -1; setting qfac to 1
pixdim[0] (qfac) should be 1 (default) or -1; setting qfac to 1
INFO:nibabel.global:pixdim[0] (qfac) should be 1 (default) or -1; setting qfac to 1
pixdim[0] (qfac) should be 1 (default) or -1; setting qfac to 1
INFO:nibabel.global:pixdim[0] (qfac) should be 1 (default) or -1; setting qfac to 1
pixdim[0] (qfac) should be 1 (default) or -1; setting qfac to 1
INFO:nibabel.global:pixdim[0] (qfac) should be 1 (default) or -1; setting qfac to 1
pixdim[0] (qfac) should be 1 (default) or -1; setting qfac to 1
INFO:nibabel.global:pixdim[0] (qfac) should be 1 (default) or -1; setting qfac to 1
pixdim[0] (qfac) should be 1 (default) or -1; setting qfac to 1
INFO:nibabel.global:pixdim[0] (qfac) should be 1 (default) or -1; setting qfac to 1
pixdim[0] (qfac) should be 1 (default) or -1; setting qfac to 1
INFO:nibabel.global:pixdim[0] (qfac) should be 1

In [16]:
vl_score_dict = {
    'Case': vl_cases,
    'Target': vl_targets,
    'Ground_Truth': vl_truth,
    'Predictions': vl_preds,
    'IOU': vl_iou_scores,
    'Precision': vl_precision_scores,
    'Recall': vl_recall_scores,
    'F1': vl_f1_scores
}

vl_score_df = pd.DataFrame(vl_score_dict)

In [17]:
vl_score_df

,Case,Target,Ground_Truth,Predictions,IOU,Precision,Recall,F1
0,d:\Github\Thesis and ML Project\Dataset\VALDO_...,84,"[91.99219, 144.72656, 152.34375, 205.66406]","[102.15552, 101.08681, 125.35493, 186.99068]",0.20905836,0.49199313,0.26660946,[[0.3458201]]
1,d:\Github\Thesis and ML Project\Dataset\VALDO_...,107,"[145.3125, 161.71875, 205.66406, 222.65625]","[66.2662, 103.6839, 127.868774, 196.1214]",0.0,0.0,0.0,0
2,d:\Github\Thesis and ML Project\Dataset\VALDO_...,104,"[88.47656, 93.75, 154.10156, 186.91406]","[76.74518, 111.84128, 121.67327, 172.80013]",0.29632834,0.73888534,0.33098936,[[0.45718098]]
3,d:\Github\Thesis and ML Project\Dataset\VALDO_...,135,"[177.53906, 151.17188, 238.47656, 212.69531]","[95.37927, 93.351105, 152.79904, 164.05623]",0.0,0.0,0.0,0
4,d:\Github\Thesis and ML Project\Dataset\VALDO_...,127,"[68.55469, 168.75, 128.32031, 229.10156]","[71.32664, 110.14809, 131.79678, 202.31863]",0.26326087,0.34326312,0.5304206,[[0.41679576]]
5,d:\Github\Thesis and ML Project\Dataset\VALDO_...,92,"[107.22656, 82.61719, 168.75, 143.55469]","[101.59649, 113.01906, 128.77806, 154.06819]",0.15643537,0.5898014,0.17553313,[[0.27054754]]
6,d:\Github\Thesis and ML Project\Dataset\VALDO_...,91,"[107.8125, 82.61719, 168.75, 144.14062]","[74.69206, 105.086174, 122.601074, 166.98611]",0.09410969,0.19475535,0.15405351,[[0.17202972]]
7,d:\Github\Thesis and ML Project\Dataset\VALDO_...,55,"[74.41406, 101.953125, 135.35156, 162.89062]","[129.54999, 168.04828, 170.50146, 182.33191]",0.0,0.0,0.0,0
8,d:\Github\Thesis and ML Project\Dataset\VALDO_...,91,"[136.52344, 76.75781, 210.9375, 145.3125]","[95.30738, 107.83768, 163.42072, 179.05437]",0.112694606,0.20779447,0.1975858,[[0.2025616]]
9,d:\Github\Thesis and ML Project\Dataset\VALDO_...,92,"[60.351562, 69.140625, 120.11719, 129.49219]","[94.92163, 123.66486, 147.57341, 182.60487]",0.022369834,0.04731184,0.040705502,[[0.043760743]]


### Metrics


In [18]:
print('Metrics for Training Set')
print(f'IOU Score: {sum(tl_iou_scores) / len(tl_iou_scores)}')
print(f'Precision Score: {sum(tl_precision_scores) / len(tl_precision_scores)}')
print(f'Recall Score: {sum(tl_recall_scores) / len(tl_recall_scores)}')
print(f'F1 Score: {sum(tl_f1_scores) / len(tl_f1_scores)}')

Metrics for Training Set
IOU Score: 0.10257294681146197
Precision Score: 0.2497207043741725
Recall Score: 0.15438712255220913
F1 Score: [[0.16359961]]


In [19]:
print('Metrics for Validation Set')
print(f'IOU Score: {sum(vl_iou_scores) / len(vl_iou_scores)}')
print(f'Precision Score: {sum(vl_precision_scores) / len(vl_precision_scores)}')
print(f'Recall Score: {sum(vl_recall_scores) / len(vl_recall_scores)}')
print(f'F1 Score: {sum(vl_f1_scores) / len(vl_f1_scores)}')

Metrics for Validation Set
IOU Score: 0.09028683578256856
Precision Score: 0.20299133099615574
Recall Score: 0.1486005645160648
F1 Score: [[0.15047953]]


#### Plotting of highest score


In [20]:
# import matplotlib.patches as patches
# import matplotlib.pyplot as plt

# num = iou_scores.index(max(iou_scores))

# slices, masks, target, case = dataset[num]

# truth_bbox = truth[num]
# predicted_bbox = preds[num]

# fig, ax = plt.subplots(1, 1, figsize=(10, 10))

# ax.imshow(slices[target][0, 0, :], cmap='gray')
# # Create the bounding box rectangle
# truth_rect = patches.Rectangle(
#     (truth_bbox[0], truth_bbox[1]),  # (x_min, y_min)
#     truth_bbox[2] - truth_bbox[0],   # width
#     truth_bbox[3] - truth_bbox[1],   # height
#     linewidth=1, edgecolor='g', facecolor='none',  # red bounding box
#     label='Truth'
# )

# # Create the bounding box rectangle
# predicted_rect = patches.Rectangle(
#     (predicted_bbox[0], predicted_bbox[1]),  # (x_min, y_min)
#     predicted_bbox[2] - predicted_bbox[0],   # width
#     predicted_bbox[3] - predicted_bbox[1],   # height
#     linewidth=1, edgecolor='r', facecolor='none',  # red bounding box
#     label='Prediction'
# )

# ax.set_title(f'IOU:{iou_scores[num]}, Precision: {precision_scores[num]}, Recall: {recall_scores[num]}, F1: {f1_scores[num]}')

# # Add the rectangle to the axis
# ax.add_patch(truth_rect)
# ax.add_patch(predicted_rect)

# ax.legend()